In [1]:
## DO NOT RUN THIS.  It is used to build an updated BkAvgAst... table which is already in the processing zip file.
## first step in creating new BkAvgAst... file
## 2017/04/07:  updated for Mar 2017 failure data (Note: only failures before 2017 used since Call data as of Dec-2016)
import numpy as np
import pandas as pd
from datetime import datetime
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

import os
os.chdir('C:/BankRisk')

df2 = pd.read_csv("df2_2001_201612.csv")
df2['AsOf'] = pd.to_datetime(df2['AsOf'])

In [2]:
fcol = df2.columns[5:]  ## get listing of numeric fields, then...
df2[fcol] = df2[fcol].astype(float)  ### CRITICAL to convert numeric data to FLOAT!!! (since some are ints)

In [3]:
df3 = df2.sort_index(by=['IDRSSD','AsOf'], ascending=True).reset_index(drop=True)
df3.head()

Unnamed: 0  IDRSSD       AsOf                    Name State     Cert  \
0           0      37 2001-03-31  BANK OF HANCOCK COUNTY    GA  10057.0   
1        8857      37 2001-06-30  BANK OF HANCOCK COUNTY    GA  10057.0   
2       17647      37 2001-09-30  BANK OF HANCOCK COUNTY    GA  10057.0   
3       26405      37 2001-12-31  BANK OF HANCOCK COUNTY    GA  10057.0   
4       35094      37 2002-03-31  BANK OF HANCOCK COUNTY    GA  10057.0   

   eTotAst   eTotLn   eALLL  eOREO   ...     ncoFIfor  ncoAg  ncoCIdom  \
0  65486.0  40507.0   979.0  200.0   ...          0.0    0.0       0.0   
1  65427.0  42153.0  1049.0    0.0   ...          0.0    0.0       0.0   
2  65575.0  44359.0  1070.0   40.0   ...          0.0    0.0       0.0   
3  66624.0  41514.0   987.0   20.0   ...          0.0    0.0       5.0   
4  68766.0  41539.0  1048.0   20.0   ...          0.0    0.0       0.0   

   ncoCIfor  ncoCard  ncoAuto  ncoOthCsmr  ncoForGovt  ncoOthLn  ncoLeas  
0       0.0      0.0      0.0        -1.0         0.0       0.0      0.0  
1       0.0      0.0      0.0       -23.0         0.0       0.0      0.0  
2       0.0      0.0      0.0         2.0         0.0       0.0      0.0  
3       0.0      0.0      0.0       104.0         0.0       0.0      0.0  
4       0.0      0.0      0.0       -13.0         0.0       0.0      0.0  

[5 rows x 194 columns]

In [4]:
len(df3) # 481187

481187

In [5]:
'eDDep' in df3.columns

True

In [6]:
# get mean eTotAst by IDRSSD:
df4 = df3[['IDRSSD','AsOf','Name','State','Cert','eTotAst','aTotAst','eTotLn','eDDep','eFDep']]
df4['eTotDep'] = df4['eDDep'] + df4['eFDep']
df5 = df4[df4['eTotAst']>0].groupby('IDRSSD')

In [7]:
df6x = pd.DataFrame(df5['AsOf'].max()).reset_index()
df6n = pd.DataFrame(df5['AsOf'].min()).reset_index()

In [8]:
df6n.tail()

IDRSSD       AsOf
10757  4368351 2012-03-31
10758  4536084 2013-12-31
10759  4569167 2013-12-31
10760  4589857 2013-12-31
10761  4845861 2015-09-30

In [9]:
df4.tail()

IDRSSD       AsOf          Name State     Cert  eTotAst  aTotAst  \
481182  4845861 2015-12-31  PRIMARY BANK    NH  59086.0  36441.0  35000.0   
481183  4845861 2016-03-31  PRIMARY BANK    NH  59086.0  44814.0  38638.0   
481184  4845861 2016-06-30  PRIMARY BANK    NH  59086.0  58096.0  49686.0   
481185  4845861 2016-09-30  PRIMARY BANK    NH  59086.0  71017.0  63658.0   
481186  4845861 2016-12-31  PRIMARY BANK    NH  59086.0  90549.0  81243.0   

         eTotLn    eDDep  eFDep  eTotDep  
481182   7683.0   8286.0    0.0   8286.0  
481183  22685.0  17242.0    0.0  17242.0  
481184  33831.0  31066.0    0.0  31066.0  
481185  53725.0  44498.0    0.0  44498.0  
481186  66353.0  64386.0    0.0  64386.0

In [10]:
df7 = df6x.merge(df4[['IDRSSD','AsOf','Name','State','Cert']],how='inner', left_on=['IDRSSD','AsOf'], right_on=['IDRSSD','AsOf'])
df7.columns = ['IDRSSD', 'xAsOf', 'xName', 'xState','xCert']
df7.head()

IDRSSD      xAsOf                             xName xState    xCert
0      37 2016-12-31            BANK OF HANCOCK COUNTY     GA  10057.0
1     242 2016-12-31  FIRST COMMUNITY BANK XENIA-FLORA     IL   3850.0
2     279 2016-12-31       MINEOLA COMMUNITY BANK, SSB     TX  28868.0
3     354 2016-12-31                  BISON STATE BANK     KS  14083.0
4     439 2007-06-30                      PEOPLES BANK     GA  16498.0

In [11]:
df7 = df7.merge(df6n[['IDRSSD','AsOf']],how='inner', left_on=['IDRSSD'], right_on=['IDRSSD'])
df7.columns = ['IDRSSD', 'xAsOf', 'xName', 'xState','xCert', 'nAsOf']
df7.tail()

IDRSSD      xAsOf                                   xName xState  \
10757  4368351 2016-12-31  KEY NATIONAL TRUST COMPANY OF DELAWARE     DE   
10758  4536084 2016-12-31                    BANK OF BIRD-IN-HAND     PA   
10759  4569167 2015-03-31                     NEW TRADITIONS BANK     FL   
10760  4589857 2014-09-30          INVESCO NATIONAL TRUST COMPANY     GA   
10761  4845861 2016-12-31                            PRIMARY BANK     NH   

         xCert      nAsOf  
10757  59069.0 2012-03-31  
10758  59074.0 2013-12-31  
10759  58822.0 2013-12-31  
10760  59078.0 2013-12-31  
10761  59086.0 2015-09-30

In [12]:
df5m = df5.mean()
df5m = df5m.reset_index()
df5m.drop('Cert', inplace=True, axis=1)
df8 = df7.merge(df5m, how='inner', left_on='IDRSSD', right_on='IDRSSD')

In [13]:
df8_orig = df8.copy()
len(df8) # 10762

10762

In [14]:
df8.head()

IDRSSD      xAsOf                             xName xState    xCert  \
0      37 2016-12-31            BANK OF HANCOCK COUNTY     GA  10057.0   
1     242 2016-12-31  FIRST COMMUNITY BANK XENIA-FLORA     IL   3850.0   
2     279 2016-12-31       MINEOLA COMMUNITY BANK, SSB     TX  28868.0   
3     354 2016-12-31                  BISON STATE BANK     KS  14083.0   
4     439 2007-06-30                      PEOPLES BANK     GA  16498.0   

       nAsOf        eTotAst        aTotAst         eTotLn          eDDep  \
0 2001-03-31   80967.718750   80664.671875   40886.796875   64550.890625   
1 2001-03-31   32938.078125   33306.156250   21181.859375   25107.718750   
2 2001-03-31  146605.953125  141592.125000   69085.515625   99852.296875   
3 2001-03-31    8933.250000    9045.125000    4013.625000    7776.687500   
4 2001-03-31  166510.615385  165717.384615  118665.038462  144540.423077   

   eFDep        eTotDep  
0    0.0   64550.890625  
1    0.0   25107.718750  
2    0.0   99852.296875  
3    0.0    7776.687500  
4    0.0  144540.423077

In [15]:
df8.columns = ['IDRSSD','xAsOf','xName','xState','xCert','nAsOf','aeTotAst','aaTotAst','aeTotLn','aeDDep','aeFDep','aeTotDep']
df8.tail()

IDRSSD      xAsOf                                   xName xState  \
10757  4368351 2016-12-31  KEY NATIONAL TRUST COMPANY OF DELAWARE     DE   
10758  4536084 2016-12-31                    BANK OF BIRD-IN-HAND     PA   
10759  4569167 2015-03-31                     NEW TRADITIONS BANK     FL   
10760  4589857 2014-09-30          INVESCO NATIONAL TRUST COMPANY     GA   
10761  4845861 2016-12-31                            PRIMARY BANK     NH   

         xCert      nAsOf       aeTotAst       aaTotAst        aeTotLn  \
10757  59069.0 2012-03-31    3694.850000    3677.500000       0.000000   
10758  59074.0 2013-12-31   98962.615385   90846.538462   82709.692308   
10759  58822.0 2013-12-31  518790.666667  518238.166667  336623.666667   
10760  59078.0 2013-12-31   97796.000000   75247.750000       0.000000   
10761  59086.0 2015-09-30   55788.000000   48332.000000   31184.000000   

              aeDDep  aeFDep       aeTotDep  
10757       0.000000     0.0       0.000000  
10758   80936.461538     0.0   80936.461538  
10759  462212.333333     0.0  462212.333333  
10760       0.000000     0.0       0.000000  
10761   28617.333333     0.0   28617.333333

In [17]:
ff = pd.read_csv("CBfails_det201703.csv")
ff2 = ff[['State', 'Year', 'Institution Name ','Cert ', 'Effective Date ']]
ff2.columns = ['State', 'Year', 'Name', 'Cert', 'EffDate']
ff2['FailDate'] = pd.to_datetime(ff2['EffDate'])
ff2 = ff2[ff2['FailDate']<'2017'] #  ONLY USE FAILDATES before 2017!!!!!!!!!!!! (to align with Call Data)
ff2.dropna(subset=['Cert'],inplace=True)
ff2.head()

State  Year                            Name     Cert    EffDate   FailDate
3    AR  2016                   ALLIED BANK       91.0  9/23/2016 2016-09-23
4    GA  2016  THE WOODBURY BANKING COMPANY    11297.0  8/19/2016 2016-08-19
5    PA  2016        FIRST CORNERSTONE BANK    35312.0   5/6/2016 2016-05-06
6    TN  2016            TRUST COMPANY BANK     9956.0  4/29/2016 2016-04-29
7    WI  2016    NORTH MILWAUKEE STATE BANK    20364.0  3/11/2016 2016-03-11

In [18]:
dff2 = ff2[ff2.duplicated(['Cert'])][['Name','Cert']]  ## find duplicate Certs
len(dff2)  # 28

28

In [19]:
ff2lst = list(dff2['Cert'])
len(ff2lst)                      ## list of dup Certs (28)

28

In [20]:
ff2a = ff2[ff2.Cert.isin(ff2lst)]
ff2a.sort_index(by=['Cert','FailDate'],inplace=True)  ## sorted list of dup Certs
ff2a.head()

State  Year                              Name    Cert     EffDate  \
1313    TX  1988        FIRST CITY, TEXAS - DALLAS  1722.0   4/20/1988   
577     TX  1992    FIRST CITY, TEXAS - DALLAS, NA  1722.0  10/30/1992   
1314    TX  1988      FIRST CITY, TEXAS - BEAUMONT  3093.0   4/20/1988   
578     TX  1992  FIRST CITY, TEXAS - BEAUMONT, NA  3093.0  10/30/1992   
1312    TX  1988         FIRST CITY, TEXAS - BRYAN  3120.0   4/20/1988   

       FailDate  
1313 1988-04-20  
577  1992-10-30  
1314 1988-04-20  
578  1992-10-30  
1312 1988-04-20

In [21]:
## sections to dedup Cert field:

In [22]:
ff2x = ff2[['Cert','FailDate']].groupby('Cert').max()
ff2x.reset_index(drop=False,inplace=True)
ff2x['mx'] = 'x'
ff2x.tail()

Cert   FailDate mx
2093  58539.0 2011-04-29  x
2094  58701.0 2013-05-10  x
2095  90027.0 1988-04-15  x
2096  90047.0 1988-12-16  x
2097  91189.0 1998-07-23  x

In [23]:
ff3 = ff2.merge(ff2x,how='left',left_on=['Cert','FailDate'],right_on=['Cert','FailDate'])
ff3.tail()

State  Year                             Name     Cert    EffDate  \
2121    MO  1970  FIRST STATE BANK OF BONNE TERRE  12227.0  8/24/1970   
2122    NJ  1970          EATONTOWN NATIONAL BANK  19276.0   8/7/1970   
2123    KY  1970       FARMERS BANK OF PETERSBURG  13164.0  6/25/1970   
2124    MI  1970   THE PEOPLES STATE SAVINGS BANK  14987.0  4/18/1970   
2125    IA  1970       STATE BANK OF PRAIRIE CITY  14571.0  2/22/1970   

       FailDate mx  
2121 1970-08-24  x  
2122 1970-08-07  x  
2123 1970-06-25  x  
2124 1970-04-18  x  
2125 1970-02-22  x

In [24]:
ff4=ff3.dropna()
len(ff3), len(ff4) # 2126, 2098

(2126, 2098)

In [25]:
df8 = df8.merge(ff4[['Cert','FailDate']], how='left', left_on='xCert', right_on='Cert')
df8 = df8.drop('Cert', axis=1)
len(df8) # 10762

10762

In [26]:
df8.sort_index(by='aeTotAst',ascending=False, inplace=True)
df8.reset_index(drop=True, inplace=True)  # renumber rows based on largest to smallest banks
df8.reset_index(drop=False, inplace=True)  # move row numbers into another column called 'index'
df8.rename(columns={'index':'Rank'},inplace=True)  # rename 'index' column to 'Rank'
df8['Rank'] = df8['Rank'] + 1  # add 1 so that largest bank has rank 1 rather than rank 0
df8.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2016-12-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2016-12-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2016-12-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2016-12-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.395950e+09  1.369624e+09  4.808213e+08  6.097738e+08   
1 2001-03-31  1.210100e+09  1.216132e+09  6.332250e+08  7.478789e+08   
2 2001-03-31  1.021323e+09  1.017492e+09  4.930308e+08  2.606809e+08   
3 2001-03-31  7.900139e+08  7.775349e+08  4.884076e+08  5.403720e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate  
0  2.526110e+08  8.623848e+08        NaT  
1  1.083941e+08  8.562730e+08 2009-01-16  
2  4.333959e+08  6.940768e+08 2008-11-23  
3  6.454305e+07  6.049150e+08        NaT  
4  2.469325e+07  2.951088e+08        NaT

In [27]:
df8['pr_eTA'] = df8['aeTotAst']*100/df8['aeTotAst'].sum()
df8['pr_eTAcs'] = df8['pr_eTA'].cumsum()
df8['pr_eTL'] = df8['aeTotLn']*100/df8['aeTotLn'].sum()
df8['pr_eTLcs'] = df8['pr_eTL'].cumsum()
df8['pr_eDD'] = df8['aeDDep']*100/df8['aeTotLn'].sum()
df8['pr_eDDcs'] = df8['pr_eDD'].cumsum()
df8.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2016-12-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2016-12-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2016-12-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2016-12-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.395950e+09  1.369624e+09  4.808213e+08  6.097738e+08   
1 2001-03-31  1.210100e+09  1.216132e+09  6.332250e+08  7.478789e+08   
2 2001-03-31  1.021323e+09  1.017492e+09  4.930308e+08  2.606809e+08   
3 2001-03-31  7.900139e+08  7.775349e+08  4.884076e+08  5.403720e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate    pr_eTA   pr_eTAcs    pr_eTL  \
0  2.526110e+08  8.623848e+08        NaT  9.155524   9.155524  5.541019   
1  1.083941e+08  8.562730e+08 2009-01-16  7.936602  17.092126  7.297330   
2  4.333959e+08  6.940768e+08 2008-11-23  6.698479  23.790605  5.681723   
3  6.454305e+07  6.049150e+08        NaT  5.181410  28.972015  5.628445   
4  2.469325e+07  2.951088e+08        NaT  2.896596  31.868612  2.934454   

    pr_eTLcs    pr_eDD   pr_eDDcs  
0   5.541019  7.027077   7.027077  
1  12.838349  8.618610  15.645687  
2  18.520072  3.004105  18.649792  
3  24.148517  6.227286  24.877077  
4  27.082971  3.116289  27.993366

In [28]:
df8.to_csv("BkAvgAstPR_FailDate_201612.csv")

In [29]:
df8_fail = df8[df8['FailDate'].map(lambda x: x.year) > 2000]   #cbt[cbt['AsOf'].map(lambda x: x.month) == 12] 
len(df8_fail) # 481

481

In [30]:
df8_fail.to_csv("FailedBkList_AvgAst_201612.csv")